In [65]:
import kfp

from typing import NamedTuple
import kfp.components as comp
from kfp import compiler, dsl
from kfp import dsl
from kfp.components import InputPath, OutputPath
from kubernetes.client.models import V1EnvVar

In [73]:
def GetMetrics_Func(name_measurement, output_timeseries_path: OutputPath(str), metrics_pipeline_path: OutputPath("Metrics")):

    # Task 1: Get Measurement Data.

    # Import Libraries
    import requests # To Requirements
    import json
    import pandas as pd # To Requirements
    import maya # To Requirements
    from tqdm import tqdm # To Requirements
    
    # Function 1: Get Request
    def GetRequest(url, headers ={}, payload = {}):
        response = requests.request("GET", url, headers = headers, data = payload)
        try:
            return response.json()
        except:
            dict_ = {
                "status_code": response.status_code,
                "text": response.text
            }
            return dict_
    
    # Function 2: Download Assets Data

    def DownloadAssetsData(measurement_name, bucket = "renergetic", min_date = "yesterday", max_date = "tomorrow"):
    
        from datetime import datetime
        

        try:
            min_date_from = maya.when(min_date).datetime()
            print("Min Date")
            print(maya.when(min_date).iso8601())
        except:
            ValueError("Please introduce correct time format for MIN_DATE")
        
        try: 
            max_date_from = maya.when(max_date).datetime()
            print("Max Date")
            print(maya.when(max_date).iso8601())
        except:
            ValueError("Please introduce correct time format for MAX_DATE")
        
        datelist = pd.date_range(min_date_from, max_date_from)
        print(list(datelist)[-1])
        data_ = []

        for i in tqdm(range(len(datelist)-1)):
            from_obj = datelist[i]
            to_obj = datelist[i+1]
            from_ = datetime.strftime(from_obj, "%Y-%m-%d 00:00:00")
            if i != (len(datelist)-1):
                to_ = datetime.strftime(to_obj, "%Y-%m-%d 00:00:00")
            else:
                to_obj = maya.now().add_minutes(15).to_datetime()
                to_ = datetime.strftime(to_obj, "%Y-%m-%d 00:00:00")
            url = "http://influx-api-ren-prototype.apps.paas-dev.psnc.pl/api/measurement/data?measurements={measurement_name}&from={from_}&to={to_}"\
                .format(measurement_name = measurement_name, from_ = from_, to_= to_)
            message = "Getting Data From {from_} to {to_}"\
                .format(from_ = from_, to_ = to_)
            print(message)
            info_ = GetRequest(url)

            if type(info_) == list:
                data_ = data_ + info_
            elif type(info_) == dict:
                print("Error")
                print(from_)
                print(to_)
        return data_
    
    # Function 3: Transform List To DataFrame:

    def DataFrameAssests(list_data, name_field):
        dicts = []
        for data in list_data:
            try:
                dict_ = {
                    "ds": data["tags"]["time_prediction"],
                    "asset_name": data["tags"]["asset_name"],
                    "direction": data["tags"]["direction"],
                    "value": float(data["fields"][name_field])
                }
                dicts.append(dict_)
            except:
                continue
        return pd.DataFrame(dicts)

    ####### EXECUTION OF CODE ############

    list_energy = DownloadAssetsData(name_measurement, min_date = "6 Oct 2022 13:15:00")

    measurements_units = {
        "heat_meter": "energy",
        "renewability": "percentage"
    }

    data_energy = DataFrameAssests(list_energy, measurements_units[name_measurement])
    print("______________________________")
    print("Maximum Date Of Data Frame")
    print(maya.MayaDT(max(data_energy.ds.apply(int))).iso8601())

    dict_output = {}
    
    for column_name in data_energy.columns.values:
        dict_output[column_name] = data_energy[column_name].tolist()
    
    with open(output_timeseries_path, "w") as out_file:
        json.dump(dict_output, out_file)
    
    try:
        numb_assets = len(list(pd.unique(data_energy["asset_name"])))
    except:
        numb_assets = 5
    numb_metrics = len(list_energy)/numb_assets

    metrics = {
        "metrics":[{
            "name": "Number Assets",
            "numberValue": numb_assets
        },
        {
            "name": "Number Metrics",
            "numberValue": numb_metrics
        }]
    }

    with open(metrics_pipeline_path, 'w') as f:
        json.dump(metrics, f)

    return numb_metrics

In [74]:
def TrainModelSMS(name_asset, input_metrics_path : InputPath(str), output_predictions_path : OutputPath(str), 
                    ml_metrics_path : OutputPath("Metrics"),split_train = 0.8, freq = 8):

    # Task 2 (Option A): Train Model and share predictions (only smoothing)
    
    # Import Libraries
    import json
    import pandas as pd # To Requirements
    from statsmodels.tsa.arima.model import ARIMA # To Requirements
    from statsmodels.tsa.holtwinters import ExponentialSmoothing # To Requirements
    from sklearn.metrics import mean_absolute_error # To Requirements
    import maya

    with open(input_metrics_path) as data_file:
        data_str = json.load(data_file)
    
    print(name_asset)

    freq = int(freq)

    ts_data = pd.DataFrame(data_str)
    ts_data = ts_data[(ts_data.asset_name == name_asset)][["value", "ds"]]
    try:
        st = float(split_train)
        n_row = int(ts_data.shape[0])
        print("Details")
        print(st)
        print(n_row)
        partition = int(st * n_row)
    except:
        partition = 200
        print("Error")
        print(split_train)
        print(ts_data.shape[0])
        

    ts = ts_data["value"].tolist()[0:partition]

    # ARIMA Model
    model = ARIMA(ts, order=(5,1,0))
    model_arima = model.fit()

    #Exponentional Smoothing Model
    model = ExponentialSmoothing(ts)
    model_expon = model.fit()

    ts_test = ts_data["value"].tolist()[partition:]

    pred_arima = model_arima.predict(partition, ts_data.shape[0]-1)
    pred_exp = model_expon.predict(partition,ts_data.shape[0]-1)
    
    mae_train_arima = mean_absolute_error(ts, model_arima.predict())
    mae_train_exp = mean_absolute_error(ts, model_arima.predict())
    mae_arima = mean_absolute_error(ts_test, pred_arima)
    mae_exp = mean_absolute_error(ts_test, pred_exp)

    if mae_arima < mae_exp:
        print("ARIMA model used for " + name_asset + " predictions")
        print(str(mae_train_arima))
        print(str(mae_arima))
        model = "ARIMA"
        mae = mae_arima
        predictions = model_arima.predict(ts_data.shape[0], ts_data.shape[0]+freq)
    else:
        print("Exponential model used for " + name_asset + " predictions")
        print(str(mae_train_exp))
        print(str(mae_exp))
        model = "Exponential Smoothing"
        mae = mae_exp
        predictions = model_expon.predict(ts_data.shape[0], ts_data.shape[0]+freq)

    
    last_date = int(ts_data["ds"].tolist()[-1])
    list_dates = []

    for i in range(freq):
        last_date = maya.MayaDT(last_date).add(minutes = 15).epoch
        list_dates.append(last_date)


    dict_output = {
        "ds": list_dates,
        "preds": list(predictions)
    }

    metrics = {
        "metrics": [{
            "mae": mae
        }]
    }


    with open(output_predictions_path, "w") as out_file:
        json.dump(dict_output, out_file)
    
    with open(ml_metrics_path, "w") as out_file:
        json.dump(metrics, out_file)



In [75]:
def TrainModelB(name_asset, input_metrics_path : InputPath(str), output_predictions_path : OutputPath(str), 
                    ml_metrics_path : OutputPath("Metrics"),split_train = 0.8, freq = 8):

    # Task 2 (Option B): Train Model and share predictions (smoothing + AutoKeras)
    
    # Import Libraries
    import json
    import pandas as pd # To Requirements
    from statsmodels.tsa.arima.model import ARIMA # To Requirements
    from statsmodels.tsa.holtwinters import ExponentialSmoothing # To Requirements
    from sklearn.metrics import mean_absolute_error # To Requirements
    import maya # To Requirements
    import autokeras as ak # To Requirements
    from tqdm import tqdm # To Requirements
    import numpy as np # To Requirements
    from discord_webhook import DiscordWebhook

    with open(input_metrics_path) as data_file:
        data_str = json.load(data_file)
    
    print(name_asset)

    freq = int(freq)

    ts_data = pd.DataFrame(data_str)
    ts_data = ts_data[(ts_data.asset_name == name_asset)][["value", "ds"]]

    print(name_asset)
    last_date = max(ts_data["ds"].apply(int))
    print(maya.MayaDT(last_date).iso8601())
    try:
        st = float(split_train)
        n_row = int(ts_data.shape[0])
        print("Details")
        print(st)
        print(n_row)
        partition = int(st * n_row)
    except:
        partition = 200
        print("Error")
        print(split_train)
        print(ts_data.shape[0])
        

    ts = ts_data["value"].tolist()[0:partition]

    # ARIMA Model
    model = ARIMA(ts, order=(5,1,0))
    model_arima = model.fit()

    #Exponentional Smoothing Model
    model = ExponentialSmoothing(ts)
    model_expon = model.fit()

    ts_test = ts_data["value"].tolist()[partition:]

    pred_arima = model_arima.predict(partition, ts_data.shape[0]-1)
    pred_exp = model_expon.predict(partition,ts_data.shape[0]-1)
    
    mae_train_arima = mean_absolute_error(ts, model_arima.predict())
    mae_train_exp = mean_absolute_error(ts, model_arima.predict())
    mae_arima = mean_absolute_error(ts_test, pred_arima)
    mae_exp = mean_absolute_error(ts_test, pred_exp)

    # Process and Train AutoKeras Model

    def PrepareAKData(data):
        if type(data) != int:
            data = data[["ds","value"]]
            list_values = []
            for index, row in tqdm(data.iterrows(), total = data.shape[0]):
                dict_ = {
                    "ds": row["ds"]
                }

                for hour in range(24):
                    if maya.MayaDT(int(row["ds"])).hour == hour:
                        dict_["hour_" + str(hour)] = True
                    else:
                        dict_["hour_" + str(hour)] = False
                
                for weekday in range(1,8):
                    if maya.MayaDT(int(row["ds"])).weekday == weekday:
                        dict_["weekday_" + str(weekday)] = True
                    else:
                        dict_["weekday_" + str(weekday)] = False
                
                for month in range(1,13):
                    if maya.MayaDT(int(row["ds"])).month == month:
                        dict_["month_" + str(month)] = True
                    else:
                        dict_["month_" + str(month)] = False
                
                dict_["value"] = row["value"]
                list_values.append(dict_)
        else:
            dict_ = {
                    "ds": data
                }

            for hour in range(24):
                if maya.MayaDT(data).hour == hour:
                    dict_["hour_" + str(hour)] = True
                else:
                    dict_["hour_" + str(hour)] = False
            
            for weekday in range(1,8):
                if maya.MayaDT(data).weekday == weekday:
                    dict_["weekday_" + str(weekday)] = True
                else:
                    dict_["weekday_" + str(weekday)] = False
            
            for month in range(1,13):
                if maya.MayaDT(data).month == month:
                    dict_["month_" + str(month)] = True
                else:
                    dict_["month_" + str(month)] = False
            list_values = [dict_]
        return pd.DataFrame(list_values)
    
    def GetPrevData(init, end, data, ts, name = "value_"):
        list_dict = []
        for i in tqdm(range(init, len(data))):
            vals = data[(i-init): (i -end)]
            dict_ = {}
            for j in range(len(vals)):
                name_val = name + str(len(vals) - j + end)
                dict_[name_val] = vals[j]
            dict_["ds"] = ts[i]
            list_dict.append(dict_)
        return pd.DataFrame(list_dict)
    try:
        train_ak = PrepareAKData(ts_data[0:partition])
        test_ak = PrepareAKData(ts_data[partition:])
    except:
        message = "There was an error while Preparing AutoKeras Data measurement {name} with name {asset}\nThe message is {text}"\
            .format(name = "heat consumption", asset = name_asset, text = "Not Available Message")
        webhook = DiscordWebhook(url='https://discord.com/api/webhooks/1002537248622923816/_9XY9Hi_mjzh2LTVqnmSKXlIFJ5rgBO2b8xna5pynUrzALgtC4aXSFq89uMdlW_v-ZzT', content=message)
        response = webhook.execute()
    try:
        prepared_data = GetPrevData(24*4, 1, ts_data["value"].tolist(), ts_data["ds"].tolist())
    except:
        message = "There was an error while Preparing Previous Data measurement {name} with name {asset}\nThe message is {text}"\
            .format(name = "heat consumption", asset = name_asset, text = "Not Available Message")
        webhook = DiscordWebhook(url='https://discord.com/api/webhooks/1002537248622923816/_9XY9Hi_mjzh2LTVqnmSKXlIFJ5rgBO2b8xna5pynUrzALgtC4aXSFq89uMdlW_v-ZzT', content=message)
        response = webhook.execute()
    train_prev = prepared_data
    test_prev = prepared_data
    


    train_data = pd.merge(train_ak, train_prev, on = "ds")
    test_data = pd.merge(test_ak, test_prev, on = "ds")
    
    print(train_data.columns.values)

    train_np = np.array(train_data.drop(["value","ds"], axis = 1))
    target_np = np.array(train_data["value"])

    test_np = np.array(test_data.drop(["value","ds"], axis = 1))
    real_np = np.array(test_data["value"])

    model_ak = ak.AutoModel(
        ak.StructuredDataInput(),
        ak.RegressionHead(),
        directory= "/tmp",
        max_trials= 4
    )
    model_ak.fit(
        train_np,
        target_np,
        epochs = 10
    )

    y_hat = model_ak.predict(test_np)
    y_hat_pred = []
    for y in y_hat:
        if pd.isna(y[0]):
            y_hat_pred.append(0)
        else:
            y_hat_pred.append(y[0])
    
    mae_autokeras = mean_absolute_error(real_np, y_hat_pred)

    if mae_arima < mae_exp:
        print("ARIMA model used for " + name_asset + " predictions")
        print(str(mae_train_arima))
        print(str(mae_arima))
        model = "ARIMA"
        mae = mae_arima
        predictions = model_arima.predict(ts_data.shape[0], ts_data.shape[0]+freq)
    else:
        print("Exponential model used for " + name_asset + " predictions")
        print(str(mae_train_exp))
        print(str(mae_exp))
        model = "Exponential Smoothing"
        mae = mae_exp
        predictions = model_expon.predict(ts_data.shape[0], ts_data.shape[0]+freq)

    
    list_dates = []

    for i in range(freq):
        last_date = maya.MayaDT(last_date).add(minutes = 15).epoch
        list_dates.append(last_date)

    if mae_autokeras < mae:
        print("AutoKeras model used for " + name_asset + " predictions")
        print(str(mae_autokeras))
        mae = mae_autokeras
        values = ts_data["value"].tolist()
        ts_ds = ts_data["ds"].tolist()
        predictions = []
        dates_predictions = []
        for date_epoch in list_dates:
            ak_data = PrepareAKData(date_epoch)
            prev_data = GetPrevData(24*4, 1, ts_data["value"].tolist() + predictions + [0,0], ts_data["ds"].tolist() + dates_predictions + [date_epoch, date_epoch + 1])
            pred_data = pd.merge(ak_data, prev_data, on = "ds")
            try:
                pred_data = pred_data.drop("value", axis = 1)
            except:
                pass
            try:
                pred_data = pred_data.drop("ds", axis = 1)
            except:
                pass
            y_hat = model_ak.predict(pred_data)
            predictions.append(float(y_hat[0][0]))
            dates_predictions.append(date_epoch)
        
        list_dates = dates_predictions





    dict_output = {
        "ds": list_dates,
        "preds": list(predictions)
    }

    metrics = {
        "metrics": [{
            "MAE": mae,
            "MAE_AUTOKERAS": mae_autokeras,
            "MAE_ARIMA": mae_arima,
            "MAE_SMOOTH": mae_exp
        }]
    }

    with open(output_predictions_path, "w") as out_file:
        json.dump(dict_output, out_file)
    
    with open(ml_metrics_path, "w") as out_file:
        json.dump(metrics, out_file)



In [76]:
def CheckModel(input_data_path: InputPath(str), numb_occurences_min: int, numb_occurences_max: int) -> bool:
    import json
    print("Minimum")
    print(numb_occurences_min)
    print("Maximum")
    print(numb_occurences_max)
    with open(input_data_path) as data_file:
        data_str = json.load(data_file)
    print("Length data")
    print(len(data_str["ds"]))

    if numb_occurences_max != 0:
        if (len(data_str) >= numb_occurences_min) and (len(data_str["ds"]) < numb_occurences_max):
            return True
        else:
            return False
    else:
        if len(data_str["ds"]) >= numb_occurences_min:
            return True
        else:
            return False
    

In [77]:
def SendPredictionsToInfluxDB(input_pred_building1_path: InputPath(str), input_pred_building2_path: InputPath(str), input_pred_sc1_path: InputPath(str)):

    import json
    import requests # TO REQUIREMENTS
    from discord_webhook import DiscordWebhook # TO REQUIREMENTS

    def GetDictPost(measurement_name, value, asset_name, time_pred):
        if measurement_name == "heat_supply":
            direction_energy = "out"
        elif measurement_name == "heat_consumption":
            direction_energy = "in"
        else:
            direction_energy = "None"
        
        data_post = {
            "bucket": "renergetic",
            "measurement": "heat_meter",
            "fields":{
                "energy": value
            },
            "tags":{
                "typeData": "forecast",
                "domain": "heat",
                "direction": direction_energy,
                "prediction_window": "2h",
                "asset_name": asset_name,
                "time_prediction": time_pred
            }
        }
            
        return data_post
    
    def SendData2Influx(measurement_name, value, asset_name, time_pred):
        import json 
        url =  "http://influx-api-ren-prototype.apps.paas-dev.psnc.pl/api/measurement"

        headers = {
        "Content-Type": "application/json"
        }

        data_post = GetDictPost(measurement_name, value, asset_name, time_pred)

        response = requests.request("POST", url, headers=headers, data=json.dumps(data_post))

        if response.status_code != 200:
            raise ValueError("Error in post \n"+ str(response.status_code) + " " + response.text)

    with open(input_pred_building1_path) as data_file:
        data_b1 = json.load(data_file)
    
    with open(input_pred_building2_path) as data_file:
        data_b2 = json.load(data_file)
    
    with open(input_pred_sc1_path) as data_file:
        data_sc1 = json.load(data_file)
    
    for i in range(len(data_b1["ds"])):
        try:
            data_post_b1 = SendData2Influx("heat_consumption", data_b1["preds"][i], "building1", data_b1["ds"][i])
        except:
            message = "There was an error while trying to upload data to Influx for measurement {name} with name {asset}\nThe message is {text}"\
                .format(name = "heat consumption", asset = "building1", text = "Not Available Message")
            webhook = DiscordWebhook(url='https://discord.com/api/webhooks/1002537248622923816/_9XY9Hi_mjzh2LTVqnmSKXlIFJ5rgBO2b8xna5pynUrzALgtC4aXSFq89uMdlW_v-ZzT', content=message)
            response = webhook.execute()
        try:
            data_post_b2 = SendData2Influx("heat_consumption", data_b2["preds"][i], "building2", data_b2["ds"][i])
        except:
            message = "There was an error while trying to upload data to Influx for measurement {name} with name {asset}\nThe message is {text}"\
                .format(name = "heat consumption", asset = "building2", text = "Not Available Message")
            webhook = DiscordWebhook(url='https://discord.com/api/webhooks/1002537248622923816/_9XY9Hi_mjzh2LTVqnmSKXlIFJ5rgBO2b8xna5pynUrzALgtC4aXSFq89uMdlW_v-ZzT', content=message)
            response = webhook.execute()
        try:
            data_post_sc1 = SendData2Influx("heat_supply", data_sc1["preds"][i], "solar_collector1", data_sc1["ds"][i])
        except:
            message = "There was an error while trying to upload data to Influx for measurement {name} with name {asset}\nThe message is {text}"\
                .format(name = "heat consumption", asset = "solar_collector1", text = "Not Available Message")
            webhook = DiscordWebhook(url='https://discord.com/api/webhooks/1002537248622923816/_9XY9Hi_mjzh2LTVqnmSKXlIFJ5rgBO2b8xna5pynUrzALgtC4aXSFq89uMdlW_v-ZzT', content=message)
            response = webhook.execute()




# PIPELINE COMPILATION

In [78]:
def REN_Forecast_Pipeline(min_value_forecast):
    env_var = V1EnvVar(name='HOME', value='/tmp')
    get_measurement_op = comp.create_component_from_func(
        GetMetrics_Func, packages_to_install = ["requests", "pandas", "maya", "tqdm"], output_component_file = "metric_op_component.yaml")
    check_op = comp.create_component_from_func(
        CheckModel, output_component_file="check_op.yaml"
    )
    TrainingModel_op = comp.create_component_from_func(
        TrainModelSMS, packages_to_install = ["requests", "pandas", "maya", "statsmodels", "sklearn"], output_component_file = "model_op_component.yaml")
    TrainingModel_B_op = comp.create_component_from_func(
        TrainModelB, packages_to_install = ["requests", "pandas", "maya", "statsmodels", "sklearn", "autokeras", "tqdm", "numpy", "discord_webhook"], output_component_file = "model_b_op_component.yaml")
    
    SendData_op = comp.create_component_from_func(
        SendPredictionsToInfluxDB, packages_to_install= ["requests", "discord_webhook"], output_component_file= "send_operation_component.yaml"
    )
    
    # Get Data from Influx Tasks
    get_heat_task = (get_measurement_op("heat_meter").add_env_variable(env_var)
                    .set_memory_request('1Gi')
                    .set_memory_limit('2Gi')
                    .set_cpu_request('1')
                    .set_cpu_limit('2'))
    
    check_A_task = (check_op(get_heat_task.outputs["output_timeseries"], 0, min_value_forecast).add_env_variable(env_var)
                    .set_memory_request('1Gi')
                    .set_memory_limit('2Gi')
                    .set_cpu_request('1')
                    .set_cpu_limit('2'))
    
    check_B_task = (check_op(get_heat_task.outputs["output_timeseries"], min_value_forecast, 0).add_env_variable(env_var)
                    .set_memory_request('1Gi')
                    .set_memory_limit('2Gi')
                    .set_cpu_request('1')
                    .set_cpu_limit('2'))

    
    #Develop and Train Model Tasks
    with dsl.Condition(check_A_task.output == True):
        modeling_b1_task = (TrainingModel_op("building1",get_heat_task.outputs["output_timeseries"]).add_env_variable(env_var).after(get_heat_task)
                            .set_memory_request('1Gi')
                            .set_memory_limit('2Gi')
                            .set_cpu_request('1')
                            .set_cpu_limit('2'))
        modeling_b2_task = (TrainingModel_op("building2",get_heat_task.outputs["output_timeseries"]).add_env_variable(env_var).after(get_heat_task)
                            .set_memory_request('1Gi')
                            .set_memory_limit('2Gi')
                            .set_cpu_request('1')
                            .set_cpu_limit('2'))
        
        modeling_sc1_task = (TrainingModel_op("solar_collector1",get_heat_task.outputs["output_timeseries"]).add_env_variable(env_var).after(get_heat_task)
                            .set_memory_request('1Gi')
                            .set_memory_limit('2Gi')
                            .set_cpu_request('2')
                            .set_cpu_limit('4'))
        
        sending_data_task = (SendData_op(modeling_b1_task.outputs["output_predictions"], modeling_b2_task.outputs["output_predictions"], modeling_sc1_task.outputs["output_predictions"]).add_env_variable(env_var)
                            .set_memory_request('1Gi')
                            .set_memory_limit('2Gi')
                            .set_cpu_request('1')
                            .set_cpu_limit('2'))
    
    with dsl.Condition(check_B_task.output == "True"):
        modeling_b_b1_task = (TrainingModel_B_op("building1",get_heat_task.outputs["output_timeseries"]).add_env_variable(env_var).after(get_heat_task)
                            .set_memory_request('1Gi')
                            .set_memory_limit('2Gi')
                            .set_cpu_request('1')
                            .set_cpu_limit('2'))
        modeling_b_b2_task = (TrainingModel_B_op("building2",get_heat_task.outputs["output_timeseries"]).add_env_variable(env_var).after(get_heat_task)
                            .set_memory_request('1Gi')
                            .set_memory_limit('2Gi')
                            .set_cpu_request('1')
                            .set_cpu_limit('2'))
        modeling_b_sc1_task = (TrainingModel_B_op("solar_collector1",get_heat_task.outputs["output_timeseries"]).add_env_variable(env_var).after(get_heat_task)
                            .set_memory_request('1Gi')
                            .set_memory_limit('2Gi')
                            .set_cpu_request('1')
                            .set_cpu_limit('2'))
        sending_b_data_task = (SendData_op(modeling_b_b1_task.outputs["output_predictions"], modeling_b_b2_task.outputs["output_predictions"], modeling_b_sc1_task.outputs["output_predictions"]).add_env_variable(env_var)
                            .set_memory_request('1Gi')
                            .set_memory_limit('2Gi')
                            .set_cpu_request('1')
                            .set_cpu_limit('2'))
    
    


compiler.Compiler().compile(pipeline_func = REN_Forecast_Pipeline, package_path ="forecast_test_Pipeline.yaml")

/Users/fablab/opt/anaconda3/lib/python3.8/site-packages/kfp/components/_data_passing.py:227: UserWarning: Missing type name was inferred as "Float" based on the value "0.8".
  warnings.warn(
/Users/fablab/opt/anaconda3/lib/python3.8/site-packages/kfp/components/_data_passing.py:227: UserWarning: Missing type name was inferred as "Integer" based on the value "8".
  warnings.warn(


In [72]:
import maya
maya.MayaDT(1666311347).iso8601()

'2022-10-21T00:15:47Z'